# GGUF Format

GGUF is the current state of the art model, started being supported for llama.cpp on August 23rd, 2023. Today is August 25th, so pretty recent, you could say! GGML is its predacessor and has since been depricated.

To use, I recommend using ctransformers, but you can also use the llama.cpp library directly as well. Be sure that you have the most updated versions, ctransformers required version >=0.2.24 for gguf support, and llama.cpp python bindings does not yet support it -- you need to be careful

In [2]:
from ctransformers import AutoModelForCausalLM
import textwrap

llm = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="msaad02/llama2_7b_brockportgpt_gguf",
    model_file="brockportgpt-7b-q4_1.gguf",
    model_type="llama"
)

Fetching 1 files: 100%|██████████| 1/1 [01:13<00:00, 73.94s/it]


In [3]:
llm_config = {
    "max_new_tokens": 256,
    "stream": True,
}

def qa(text: str, full = False):
    # textwrap.dedent gets rid of indenting at the start of each newline
    text = textwrap.dedent(f"""\
        Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

        ### Instruction:
        {text}

        ### Response:
        """)
    
    generator = llm(text, **llm_config)

    # print beginning?
    if full:
        print(text, end="")

    response = ""
    for token in generator:
        response = response + token
        print(token, end="")

    response = (text + response) if full else response

    return response

In [4]:
output = qa("How do I apply?", full=True)

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
How do I apply?

### Response:
To apply to the Chemistry program at SUNY Brockport, you can visit their website and follow the application process provided there.

In [5]:
output = qa("is there a CS department?")

Yes, the University offers both the undergraduate & graduate courses and certificate in Data Science with a concentration in cybersecurity.

In [6]:
output = qa("Tell me about the CS department at Brockport.")

# yikes!! Didn't follow the format perfectly, and kept going. No good.

The CS department offers a variety of programs that meet the needs of different students. The website provides information on their degree programs, course offerings, student success stories, and career opportunities in the field of Computer Science.

In [7]:
def sample_qa(text: str, full = False, n = 3):
    """
    For a given question, run the model n times to see a variety of responses.
    """
    answer_list = []

    print("1. ", end="")
    for num in range(n):
        answer_list.append(qa(text, full))

        if num+1 < n:
            print(f"\n\n{num+2}. ", end="")

    return answer_list

In [8]:
output = sample_qa("Tell me about the CS department at Brockport.")

1. The CS department at Brockport offers a wide range of programs that equip students with the skills needed for success in various industries.

2. The CS department at Brockport provides a wide range of education for students interested in computer science.

3. The CS department at Brockport provides opportunities for students to pursue their interests in computer science. It is a dynamic and ever-growing field, and Brockport offers both general courses as well as those designed to address industry standards.

In [9]:
output = sample_qa("Im an undergrad. What are the steps to apply?")

1. The application process for undergraduate students at SUNY Brockport includes submitting relevant supporting documentation, paying the application fee, and completing any required sections within WebBanner, including the Student Information Form (SIF).

2. The steps to apply as an undergraduate include submitting your application on their website, paying the $50 non-refundable application fee, requesting transcripts and documents, meeting any prerequisites for your major, and completing any required tests.

3. To apply as an undergraduate at SUNY Brockport, you must submit your application through the Common Application or by mail with a nonrefundable $50 fee and official high school transcripts.